In this notebook we create few plots of wind intensity measured by satellite, with a particular interest on the Canary Islands, which frequently generate the so-called *von Karman* vortex street.

In [3]:
import datetime
import re
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import filament
import logging
import copy
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.mpl.ticker as cartopyticker
myproj = ccrs.PlateCarree()
coast = cfeature.GSHHSFeature(scale="f")
from importlib import reload
reload(filament)

<module 'filament' from '/home/ctroupin/Projects/FNRS-UpwellingFilaments/Filament-Submesoscale/python/filament.py'>

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.info("Starting")

INFO:root:Starting


## Input parameters
We set the date of interest and the domain coordinates are taken from the geoTIFF file.

In [40]:
year, month, day = 2020, 4, 15
domain = [6, 10.5, 41., 44.5]
figdir = "../figures/wind/Corsica"
visfile = "/data/Visible/Sentinel-3_OLCI_2020-04-15.tiff"

In [30]:
#os.path.isdir(visibledir)
vis = filament.Visible()
vis.read_geotiff(visfile)
if not(os.path.exists(figdir)):
    os.mkdir(figdir)

## Data

In [31]:
datewind = datetime.datetime(year, month, day)
dayofyear = datewind.timetuple().tm_yday

In [32]:
extent2 = copy.deepcopy(domain)
extent2[0] -= 360.
extent2[1] -= 360.

### Scatterometer wind
The wind (netCDF) are read from the JPL OPEnDAP server.      
The files are sorted by years and days of year.

With these pieces of information we can create a list of files URLs.

In [21]:
filelist1 = filament.get_filelist_url(year, dayofyear)
filelistclean = []

INFO:Filament:Found 29 files


In [25]:
filelist2 = sorted(glob.glob(os.path.join("/data/Wind/ASCAT/Corsica/", "ascat_20200415*.nc.gz")))
filelist2

['/data/Wind/ASCAT/Corsica/ascat_20200415_080900_metopb_39306_eps_o_coa_3202_ovw.l2.nc.gz',
 '/data/Wind/ASCAT/Corsica/ascat_20200415_083600_metopa_69993_eps_o_coa_3202_ovw.l2.nc.gz',
 '/data/Wind/ASCAT/Corsica/ascat_20200415_184200_metopa_69999_eps_o_coa_3202_ovw.l2.nc.gz',
 '/data/Wind/ASCAT/Corsica/ascat_20200415_195700_metopb_39313_eps_o_coa_3202_ovw.l2.nc.gz']

In [41]:
for ifile, dataurl in enumerate(filelist2):

    logger.info("Working on {}".format(dataurl))
    wind = filament.Wind()
    res = wind.read_ascat(dataurl, extent2)
        
    if res:
        filelistclean.append(dataurl)
        mm = str(month).zfill(2)
        dd = str(day).zfill(2)
        
        fname = "".join(("wind-vis-", dataurl.split("/")[-1].split(".")[0], ".jpg"))
        logger.info("Saving figure as {}".format(fname))
        figname = os.path.join(figdir, fname)
        
        fig = plt.figure(figsize=(8, 8))
        ax = plt.subplot(111, projection=myproj)
        
        # Check if there is a visible background
        if vis.lon is not None:
            vis.add_to_plot(ax, myproj)
            
        wind.add_to_plot(fig, ax, domain=domain,
                         clim=[0, 16], vis=True, quivscale=500, quivwidth=0.001,
                         date=wind.time.strftime("%Y-%m-%d %H:%M:%S"),
                         cbarloc='upper right')
        
        
        filament.decorate_map(ax, domain, 
                              np.arange(6, 12., 1.), 
                              np.arange(41., 44.5, 0.5))
        
        #ax.add_feature(coast, color=".6", edgecolor="k", zorder=5)

        
        #plt.show()
        plt.savefig(figname, dpi=300, bbox_inches="tight")
        plt.close()
logger.info("Figures printed in {}".format(figdir))

INFO:root:Working on /data/Wind/ASCAT/Corsica/ascat_20200415_080900_metopb_39306_eps_o_coa_3202_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as wind-vis-ascat_20200415_080900_metopb_39306_eps_o_coa_3202_ovw.jpg
INFO:root:Working on /data/Wind/ASCAT/Corsica/ascat_20200415_083600_metopa_69993_eps_o_coa_3202_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as wind-vis-ascat_20200415_083600_metopa_69993_eps_o_coa_3202_ovw.jpg
INFO:root:Working on /data/Wind/ASCAT/Corsica/ascat_20200415_184200_metopa_69999_eps_o_coa_3202_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as wind-vis-ascat_20200415_184200_metopa_69999_eps_o_coa_3202_ovw.jpg
INFO:root:Working on /data/Wind/ASCAT/Corsica/ascat_20200415_195700_metopb_39313_eps_o_coa_3202_ovw.l2.nc.gz
INFO:Filament:Subsetting data to region of interest
INFO:root:Saving figure as wind-vis-ascat_20200415_195700_metopb_39313_eps